Я решаю данную задачу используя библиотеку LightAutoML [ссылка](https://github.com/sb-ai-lab/LightAutoML). 

# Условия

## Описание задачи
В вашем распоряжении есть набор изображений, которые состоят из нескольких видов геометрических фигур:

квадратов;
прямоугольников;
параллелограммов;
окружностей.
Ваша задача – посчитать количество квадратов на каждом изображении из тестовых данных. Изображения для обучения можно скачать здесь.

## Описание данных
Весь набор данных состоит из 3 типов изображений в равном количестве:

Содержат только квадраты, которые могут иметь общие стороны, вкладываться друг в друга, но не пересекаться.
Содержат квадраты и другие фигуры, все они могут иметь общие стороны, вкладываться друг в друга, но не пересекаться.
Содержат квадраты и другие фигуры, которые дополнительно могут пересекаться.
Ниже находятся 2 файла, которые потребуются для решения задачи: train.csv – содержит информацию об изображениях для обучения, test.csv – содержит информацию об изображениях для теста. Имеют поля:

* img_path – путь до изображения
* label – число квадратов на изображении (только в обучающем наборе)
* type – тип изображения 1/2/3 (только в обучающем наборе)


Пример сабмита можно посмотреть в файле sample_submission.csv. Он содержит одну колонку label, порядок соответствует порядку данных в тестовом наборе.

## Метрика качества
В качестве метрики качества используется метрика Root Mean Square Error (RMSE):

## 0. Предварительная подготовка

### 0.0. Установим LightAutoML

In [1]:
# установим библиотеку LightAutoML с поддержкой энкодеров
# для текста и изображений на основе нейросетей
# !pip install -U lightautoml[all]

In [2]:
# чтобы работали нейросети и шаблон решения задач
# с картинками нужно поставить на библиотеку дополнительный фикс!
# !pip install LightAutoML-0.3.7-py3-none-any.whl[all]

In [3]:
# в качестве модели извлекающей признаки из картинок будет
# использоваться нейросеть EfficientNet
# установим эту зависимость.

# !pip install efficientnet-pytorch==0.7.0

# QUICK WORKAROUND FOR PROBLEM WITH PANDAS
# !pip install -U pandas

In [4]:
import os
import time

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import matplotlib.pyplot as plt

# LightAutoML шаблоны пресетов для задач комп зрения
from lightautoml.automl.presets.image_presets import TabularCVAutoML
from lightautoml.tasks import Task

/home/admin-gpu/anaconda3/envs/sber_lama/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
np.random.seed(42)
torch.set_num_threads(2)

### Загрузка данных

In [6]:
train = pd.read_csv('data/train.csv')

In [7]:
train.head()

,img_path,label,type
0,train_data/1.png,3,1
1,train_data/2.png,7,3
2,train_data/3.png,9,2
3,train_data/4.png,9,1
4,train_data/5.png,10,1


In [8]:
# добавим полный относительный путь до картинок из train
# в качестве новой колонки
path_to_dataset_train = 'data'
train['path'] =  path_to_dataset_train + '/' + train['img_path']

In [9]:
# колонка с имена картинок больше не нужна
train.drop(columns=['img_path'], inplace=True)

In [10]:
train.head()

,label,type,path
0,3,1,data/train_data/1.png
1,7,3,data/train_data/2.png
2,9,2,data/train_data/3.png
3,9,1,data/train_data/4.png
4,10,1,data/train_data/5.png


In [11]:
# посмотрим на типы картинок и их количество
np.unique(train['type'].to_numpy(), return_counts=True)

(array([1, 2, 3]), array([1000, 1000, 1000]))

# Ставим задачу

In [66]:
train_1 = train[train['type']==3]
train_1

,label,type,path
1,7,3,data/train_data/2.png
5,8,3,data/train_data/6.png
6,2,3,data/train_data/7.png
9,4,3,data/train_data/10.png
10,8,3,data/train_data/11.png
...,...,...,...
2977,9,3,data/train_data/2978.png
2979,2,3,data/train_data/2980.png
2986,3,3,data/train_data/2987.png
2989,7,3,data/train_data/2990.png


In [67]:
# у нас задача регрессии, ставим метрику и лосс согласно условиям
# соревнования
task = Task('reg', loss='mse', metric='mse')

In [68]:
# целевая переменная для регрессии это
# переменная пути для картинок в столбце path
roles = {'target': 'label',
         'path' : ['path'],
         'drop' : ['type']
         }

In [69]:
automl = TabularCVAutoML(task=task,
                         timeout=5 * 3600,
                         reader_params={'cv': 5, 'random_state': 42})


# Обучение модели по кросс валидации по всей выборке

In [70]:
%%time 
# будет использовано 5 фолдов кросс валидации
# для извлечения признаков с картинок будет использована нейросеть EfficientNet b0
# после извлечения числовых признаков нейросетью на них будут учится стандартные модели
#  линейная регрессия и градиентный бустинг
oof_pred = automl.fit_predict(train_1, roles = roles, verbose = 4)

[17:07:45] Stdout logging level is DEBUG.
[17:07:45] Task: reg

[17:07:45] Start automl preset with listed constraints:
[17:07:45] - time: 18000.00 seconds
[17:07:45] - CPU: 4 cores
[17:07:45] - memory: 16 GB

[17:07:45] Train data shape: (1000, 3)

[17:07:45] Layer 1 train process start. Time left 18000.00 secs
Loaded pretrained weights for efficientnet-b0


100%|██████████| 8/8 [00:01<00:00,  7.15it/s]

[17:07:46] Feature path transformed
[17:07:46] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[17:07:46] Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 1280}
[17:07:46] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[17:07:46] Linear model: C = 1e-05 score = -3.290336355995413
[17:07:46] Linear model: C = 5e-05 score = -2.5813180813072107
[17:07:46] Linear model: C = 0.0001 score = -2.3822547395567812


[17:07:46] Linear model: C = 0.0005 score = -2.2351748387457704
[17:07:46] Linear model: C = 0.001 score = -2.340700139572909
[17:07:46] Linear model: C = 0.005 score = -3.054425458014818
[17:07:46] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[17:07:46] Linear model: C = 1e-05 score = -2.7223083745390766
[17:07:46] Linear model: C = 5e-05 score = -2.102641636078068
[17:07:46] Linear model: C = 0.0001 score = -1.9351973063468333
[17:07:46] Linear model: C = 0.0005 score = -1.7734686012504381
[17:07:46] Linear model: C = 0.001 score = -1.88532677609632
[17:07:47] Linear model: C = 0.005 score = -2.7145224444802922
[17:07:47] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[17:07:47] Linear model: C = 1e-05 score = -3.451708162097558
[17:07:47] Linear model: C = 5e-05 score = -2.747778888255939
[17:07:47] Linear model: C = 0.0001 score = -2.552185035404095
[17:07:47] Linear model: C = 0.0005 score = -2.338828990753453
[17:07:47] Linear model

[17:07:48] 200:	learn: 1.1387606	test: 1.9424722	best: 1.9424722 (200)	total: 195ms	remaining: 1.74s
[17:07:48] 300:	learn: 0.8708839	test: 1.9461138	best: 1.9388665 (214)	total: 293ms	remaining: 1.66s
[17:07:48] 400:	learn: 0.6772107	test: 1.9459783	best: 1.9388665 (214)	total: 392ms	remaining: 1.56s
[17:07:49] 500:	learn: 0.5279546	test: 1.9486804	best: 1.9388665 (214)	total: 489ms	remaining: 1.46s
[17:07:49] bestTest = 1.938866469
[17:07:49] bestIteration = 214
[17:07:49] Shrink model to first 215 iterations.
[17:07:49] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


[17:07:49] 0:	learn: 2.8407705	test: 2.8971030	best: 2.8971030 (0)	total: 1.36ms	remaining: 2.73s
[17:07:49] 100:	learn: 1.5454810	test: 1.8191899	best: 1.8191899 (100)	total: 98.9ms	remaining: 1.86s
[17:07:49] 200:	learn: 1.1413926	test: 1.7539781	best: 1.7538285 (195)	total: 196ms	remaining: 1.76s
[17:07:49] 300:	learn: 0.8589727	test: 1.7588082	best: 1.7486629 (244)	total: 295ms	remaining: 1.66s
[17:07:49] 400:	learn: 0.6568455	test: 1.7716631	best: 1.7486629 (244)	total: 393ms	remaining: 1.56s
[17:07:49] 500:	learn: 0.5104007	test: 1.7784977	best: 1.7486629 (244)	total: 491ms	remaining: 1.47s
[17:07:49] bestTest = 1.748662906
[17:07:49] bestIteration = 244
[17:07:49] Shrink model to first 245 iterations.
[17:07:49] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


[17:07:50] 0:	learn: 2.8460079	test: 2.8769214	best: 2.8769214 (0)	total: 1.31ms	remaining: 2.63s
[17:07:50] 100:	learn: 1.4904968	test: 2.1233118	best: 2.1231674 (99)	total: 99.1ms	remaining: 1.86s
[17:07:50] 200:	learn: 1.1045177	test: 2.0984695	best: 2.0973912 (196)	total: 197ms	remaining: 1.76s
[17:07:50] 300:	learn: 0.8352560	test: 2.0932681	best: 2.0900043 (286)	total: 296ms	remaining: 1.67s
[17:07:50] 400:	learn: 0.6437063	test: 2.0935327	best: 2.0889738 (334)	total: 394ms	remaining: 1.57s
[17:07:50] 500:	learn: 0.5052103	test: 2.0882093	best: 2.0876403 (497)	total: 491ms	remaining: 1.47s
[17:07:50] 600:	learn: 0.3959530	test: 2.0843268	best: 2.0835877 (595)	total: 590ms	remaining: 1.37s
[17:07:50] 700:	learn: 0.3127334	test: 2.0833182	best: 2.0827855 (630)	total: 690ms	remaining: 1.28s
[17:07:51] 800:	learn: 0.2561483	test: 2.0816310	best: 2.0799939 (780)	total: 788ms	remaining: 1.18s
[17:07:51] 900:	learn: 0.2048125	test: 2.0806922	best: 2.0799939 (780)	total: 886ms	remaining:

[17:07:51] 0:	learn: 2.8511823	test: 2.8705065	best: 2.8705065 (0)	total: 1.43ms	remaining: 2.86s
[17:07:51] 100:	learn: 1.5442136	test: 1.8975697	best: 1.8975697 (100)	total: 97.9ms	remaining: 1.84s
[17:07:51] 200:	learn: 1.1488384	test: 1.8539256	best: 1.8539256 (200)	total: 195ms	remaining: 1.74s
[17:07:52] 300:	learn: 0.8631894	test: 1.8494197	best: 1.8416669 (270)	total: 292ms	remaining: 1.65s
[17:07:52] 400:	learn: 0.6709192	test: 1.8492551	best: 1.8416669 (270)	total: 389ms	remaining: 1.55s
[17:07:52] 500:	learn: 0.5263903	test: 1.8535426	best: 1.8416669 (270)	total: 486ms	remaining: 1.46s
[17:07:52] bestTest = 1.841666924
[17:07:52] bestIteration = 270
[17:07:52] Shrink model to first 271 iterations.
[17:07:52] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


[17:07:52] 0:	learn: 2.8442878	test: 2.8909451	best: 2.8909451 (0)	total: 1.35ms	remaining: 2.71s
[17:07:52] 100:	learn: 1.4900727	test: 2.0934621	best: 2.0892267 (95)	total: 98.6ms	remaining: 1.85s
[17:07:52] 200:	learn: 1.0965599	test: 2.0432992	best: 2.0369077 (191)	total: 197ms	remaining: 1.76s
[17:07:52] 300:	learn: 0.8301084	test: 2.0207028	best: 2.0207028 (300)	total: 294ms	remaining: 1.66s
[17:07:53] 400:	learn: 0.6419379	test: 2.0075994	best: 2.0069949 (392)	total: 392ms	remaining: 1.56s
[17:07:53] 500:	learn: 0.5046097	test: 2.0145593	best: 2.0068225 (401)	total: 489ms	remaining: 1.46s
[17:07:53] 600:	learn: 0.4029887	test: 2.0251918	best: 2.0068225 (401)	total: 587ms	remaining: 1.37s
[17:07:53] 700:	learn: 0.3231427	test: 2.0255083	best: 2.0068225 (401)	total: 684ms	remaining: 1.27s
[17:07:53] bestTest = 2.006822546
[17:07:53] bestIteration = 401
[17:07:53] Shrink model to first 402 iterations.
[17:07:53] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -3.7124945518743

In [71]:
# смотрим предсказания модели-регрессора
preds = oof_pred.data[:,0]
preds


array([ 7.664083  ,  7.7440004 ,  2.3713226 ,  3.9459362 ,  6.910086  ,
        0.40639082,  4.493323  ,  3.3295693 ,  3.1708148 ,  3.1915648 ,
        5.547356  ,  2.592135  ,  4.403446  ,  3.739616  ,  3.1468477 ,
        7.621834  ,  1.4863486 ,  7.112201  ,  7.1521473 ,  6.520759  ,
        7.8669033 ,  7.055056  ,  5.659479  ,  3.286253  ,  8.055955  ,
        4.3486867 ,  6.7009993 ,  7.1216116 ,  6.8271832 ,  1.5341389 ,
        3.1701941 ,  3.1913252 ,  1.6448872 ,  6.114487  ,  4.1095915 ,
        5.250565  ,  7.817989  ,  1.5435674 ,  3.4862387 ,  0.4698999 ,
        5.531953  , -0.77084833,  4.252273  ,  8.381791  ,  2.732373  ,
        5.9609213 ,  5.051256  ,  6.7544384 ,  3.5900254 ,  3.0431666 ,
        6.5449176 ,  6.5057974 ,  3.6035314 ,  2.4260147 ,  7.672275  ,
        2.6026525 ,  3.0143373 ,  4.03655   ,  9.058183  ,  3.7983487 ,
        0.9272416 ,  8.509907  ,  6.8172045 ,  4.276464  ,  3.891029  ,
        0.0423829 , -0.07931858,  1.7261205 ,  8.535132  ,  7.47

In [72]:
y_true = train_1['label'].values
y_true

array([ 7,  8,  2,  4,  8,  0,  4,  5,  3,  4,  3,  0,  2,  2,  3,  5,  3,
        9,  9,  8,  8, 10,  8,  4,  8,  4,  8,  7,  9,  0,  2,  2,  1,  8,
        7,  3,  9,  2,  3,  0,  4,  0,  4,  9,  2,  8,  3,  7,  2,  2,  6,
        4,  2,  3, 10,  2,  4,  5, 10,  4,  1,  9,  5,  5,  1,  0,  2,  2,
       10,  8,  2,  1,  5,  5,  1,  7,  6,  7,  3,  2,  9,  7,  4,  1,  2,
        9,  2,  9,  4,  2,  6,  9,  5,  6,  6,  7,  9,  5,  4,  0,  9, 10,
        4,  9,  4,  5,  4,  7,  6,  2,  6,  7,  4,  8,  7,  3,  8,  6,  9,
        2,  6,  6,  9,  1,  4,  3,  5,  2,  3,  1,  8,  7,  6,  5,  6,  2,
        7,  4,  6,  6,  6,  2,  6, 10,  9,  9,  2,  2, 10,  3,  5, 10,  2,
        2, 10,  8, 10,  5,  5,  5,  2,  9,  5,  1,  9,  1,  1,  8,  2,  9,
        7,  8,  2,  9,  7,  6,  6,  8,  6,  0,  2,  8,  2,  4,  8,  5,  5,
        7,  4,  7,  6,  6,  7,  5,  1,  5,  1,  8,  6,  5,  3,  4,  9,  7,
        2,  8,  4,  3, 10, 10, 10,  5,  5,  4,  1,  3,  5,  4,  9,  4,  6,
        0,  6,  3,  9,  9

In [73]:
len(y_true), len(preds)

(1000, 1000)

In [74]:
# Введем метрики оценки качества модели
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()
def mae(y_true, y_pred):
    return (np.abs(y_true - y_pred)).mean()

In [75]:
# Оценим нашу модель на валидационной выборке:
print(f"MSE = {mse(y_true, preds)}")
print(f"MAE = {mae(y_true, preds)}")

# все класс 1
# MSE = 0.34593320973622943
# MAE = 0.45251106303930283

# все класс 2
# MSE = 1.9758710243245299
# MAE = 1.1024402384161949

# все класс 3
# MSE = 2.1170836277245817
# MAE = 1.157314394492656

# все и все классы сразу
# MSE = 1.5963662768334919
# MAE = 0.9565228770921628

MSE = 2.1170836277245817
MAE = 1.157314394492656


# Предскажем на тесте

In [23]:
test = pd.read_csv('data/test.csv')

In [24]:
test.head()

,img_path
0,test_data/1.png
1,test_data/2.png
2,test_data/3.png
3,test_data/4.png
4,test_data/5.png


In [26]:
# добавим полный относительный путь до картинок из test
# в качестве новой колонки
path_to_dataset_test = 'data'
test['path'] =  path_to_dataset_test + '/' + test['img_path']

In [27]:
# колонка с имена картинок больше не нужна
test.drop(columns=['img_path'], inplace=True)

In [28]:
test.head()

,path
0,data/test_data/1.png
1,data/test_data/2.png
2,data/test_data/3.png
3,data/test_data/4.png
4,data/test_data/5.png


In [29]:
# предсказание ансамблем моделей на тестовых данных
test_pred = automl.predict(test)

100%|██████████| 47/47 [00:04<00:00,  9.54it/s]

[08:07:43] Feature path transformed



/home/admin-gpu/anaconda3/envs/sber_lama/lib/python3.9/site-packages/lightautoml/image/image.py:84: RuntimeWarning: invalid value encountered in divide
  return list(hist / hist.sum())


In [30]:
test_pred = test_pred.data[:,0]

In [31]:
test_pred

array([ 7.4513717, 11.071525 ,  1.4318794, ..., 11.1494255,  6.2699213,
        3.389724 ], dtype=float32)

# публикуем сабмит

In [32]:
submission = pd.read_csv('data/sample_submission.csv')

In [33]:
submission

,label
0,0
1,0
2,0
3,0
4,0
...,...
5995,0
5996,0
5997,0
5998,0


In [34]:
submission['label'] = test_pred


In [35]:
submission

,label
0,7.451372
1,11.071525
2,1.431879
3,6.963997
4,7.484351
...,...
5995,10.608534
5996,9.632649
5997,11.149426
5998,6.269921


In [36]:
submission_path = 'submission/lightautoml.csv'

In [37]:
submission.to_csv(submission_path, index=False)